---
layout: post  
---

The goal of this post is to re-create the NCBI taxonomy database in Neo4J

In [109]:
if !occursin("cypher-shell", ENV["PATH"])
    ENV["PATH"] = "/Users/cameronprybol/Software/cypher-shell:$(ENV["PATH"])"
end

"/Users/cameronprybol/Software/cypher-shell:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Users/cameronprybol/miniconda/bin:/Users/cameronprybol/.gem/ruby/2.6.0/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin"

In [115]:
import Pkg
pkgs = [
    "DataFrames",
    "ProgressMeter",
    "LightGraphs",
    "MetaGraphs",
    "GraphPlot",
    "GraphIO",
    "EzXML",
    "ParserCombinator",
    "uCSV"
]

for pkg in pkgs
    try
        Pkg.add(pkg)
    catch
#         # tried to install an unregistered local package
    end
    eval(Meta.parse("import $pkg"))
end

using EzXML
using ParserCombinator

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/

In [3]:
TASK = "ncbi-taxonomy"
DATE = "2021-03-21"
DIR = "$(homedir())/$(DATE)-$(TASK)"
if !isdir(DIR)
    mkdir(DIR)
end
cd(DIR)

In [4]:
taxdump_url = "https://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz"
# if !isfile("$(DIR)/$(basename(taxdump_url)")

"https://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz"

In [5]:
taxdump_local_tarball = "$(DIR)/$(basename(taxdump_url))"

"/Users/cameronprybol/2021-03-21-ncbi-taxonomy/taxdump.tar.gz"

In [6]:
if !isfile(taxdump_local_tarball)
    download(taxdump_url, taxdump_local_tarball)
end

In [8]:
taxdump_out = replace(taxdump_local_tarball, ".tar.gz" => "")
if !isdir(taxdump_out)
    mkpath(taxdump_out)
    run(`tar -xvzf $(taxdump_local_tarball) -C $(taxdump_out)`)
end

In [9]:
readdir(taxdump_out)

10-element Array{String,1}:
 ".ipynb_checkpoints"
 "citations.dmp"
 "delnodes.dmp"
 "division.dmp"
 "gc.prt"
 "gencode.dmp"
 "merged.dmp"
 "names.dmp"
 "nodes.dmp"
 "readme.txt"

In [10]:
# Taxonomy names file (names.dmp):
# 	tax_id					-- the id of node associated with this name
# 	name_txt				-- name itself
# 	unique name				-- the unique variant of this name if name not unique
# 	name class				-- (synonym, common name, ...)

names_dmp = DataFrames.DataFrame(
    tax_id = Int[],
    name_txt = String[],
    unique_name = String[],
    name_class = String[]
)
ProgressMeter.@showprogress for line in split(read(open("$(taxdump_out)/names.dmp"), String), "\t|\n")
    if isempty(line)
        continue
    else
        (tax_id_string, name_txt, unique_name, name_class) = split(line, "\t|\t")
        tax_id = parse(Int, tax_id_string)
        row = (;tax_id, name_txt, unique_name, name_class)
        push!(names_dmp, row)
    end
end
names_dmp

Progress: 100%|█████████████████████████████████████████| Time: 0:00:1639m


,tax_id,name_txt
,Int64,String
1,1,all
2,1,root
3,2,Bacteria
4,2,bacteria
5,2,eubacteria
6,2,Monera
7,2,Procaryotae
8,2,Prokaryotae
9,2,Prokaryota


In [11]:
unique_tax_ids = unique(names_dmp[!, "tax_id"])

2317173-element Array{Int64,1}:
       1
       2
       6
       7
       9
      10
      11
      13
      14
      16
      17
      18
      19
       ⋮
 2820007
 2820021
 2820040
 2820056
 2820085
 2820136
 2820147
 2820189
 2820209
 2820263
 2820280
 2820281

In [177]:
ncbi_taxonomy = MetaGraphs.MetaDiGraph(length(unique_tax_ids))
ProgressMeter.@showprogress for (index, group) in enumerate(collect(DataFrames.groupby(names_dmp, "tax_id")))
    MetaGraphs.set_prop!(ncbi_taxonomy, index, :tax_id, group[1, "tax_id"])
    for row in DataFrames.eachrow(group)
        unique_name = isempty(row["unique_name"]) ? row["name_txt"] : row["unique_name"]
        # remove quotes since neo4j doesn't like them
        unique_name = replace(unique_name, '"' => "")
        # replace spaces and dashes with underscores
        name_class = Symbol(replace(replace(row["name_class"], r"\s+" => "-"), "-" => "_"))
#         name_class = Symbol(row["name_class"])
        if haskey(MetaGraphs.props(ncbi_taxonomy, index), name_class)
            current_value = MetaGraphs.get_prop(ncbi_taxonomy, index, name_class)
            if (current_value isa Array) && !(unique_name in current_value)
                new_value = [current_value..., unique_name]
                MetaGraphs.set_prop!(ncbi_taxonomy, index, name_class, new_value)
            elseif !(current_value isa Array) && (current_value != unique_name)
                new_value = [current_value, unique_name]
                MetaGraphs.set_prop!(ncbi_taxonomy, index, name_class, new_value)
            else
                continue
            end
        else
            MetaGraphs.set_prop!(ncbi_taxonomy, index, name_class, unique_name)
        end
    end
end

divisions = Dict()
for line in split(read(open("$(taxdump_out)/division.dmp"), String), "\t|\n")
    if !isempty(line)
        (id_string, shorthand, full_name, notes) = split(line, "\t|\t")
        id = parse(Int, id_string)
        divisions[id] = Dict(:division_cde => shorthand, :division_name => full_name)
    end
end
node_2_taxid_map = map(index -> ncbi_taxonomy.vprops[index][:tax_id], LightGraphs.vertices(ncbi_taxonomy))
ProgressMeter.@showprogress for line in split(read(open("$(taxdump_out)/nodes.dmp"), String), "\t|\n")
    if isempty(line)
        continue
    else
        (tax_id_string, parent_tax_id_string, rank, embl_code, division_id_string) = split(line, "\t|\t")
        
        
        division_id = parse(Int, division_id_string)
        
        tax_id = parse(Int, tax_id_string)
        lightgraphs_tax_ids = searchsorted(node_2_taxid_map, tax_id)
        @assert length(lightgraphs_tax_ids) == 1
        lightgraphs_tax_id = first(lightgraphs_tax_ids)
        
        parent_tax_id = parse(Int, parent_tax_id_string)
        lightgraphs_parent_tax_ids = searchsorted(node_2_taxid_map, parent_tax_id)
        @assert length(lightgraphs_parent_tax_ids) == 1
        lightgraphs_parent_tax_id = first(lightgraphs_parent_tax_ids)
        
        LightGraphs.add_edge!(ncbi_taxonomy, lightgraphs_tax_id, lightgraphs_parent_tax_id)
        MetaGraphs.set_prop!(ncbi_taxonomy, lightgraphs_tax_id, :rank, rank)
        # these should probably be broken out as independent nodes!
        MetaGraphs.set_prop!(ncbi_taxonomy, lightgraphs_tax_id, :division_id, division_id)
        MetaGraphs.set_prop!(ncbi_taxonomy, lightgraphs_tax_id, :division_cde, divisions[division_id][:division_cde])
        MetaGraphs.set_prop!(ncbi_taxonomy, lightgraphs_tax_id, :division_name, divisions[division_id][:division_name])
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:04:04
Progress: 100%|█████████████████████████████████████████| Time: 0:01:49


In [178]:
collect(LightGraphs.edges(ncbi_taxonomy))

2317173-element Array{LightGraphs.SimpleGraphs.SimpleEdge{Int64},1}:
 Edge 1 => 1
 Edge 2 => 100805
 Edge 3 => 280007
 Edge 4 => 3
 Edge 5 => 14946
 Edge 6 => 1384594
 Edge 7 => 1338
 Edge 8 => 165190
 Edge 9 => 8
 Edge 10 => 14787
 Edge 11 => 10
 Edge 12 => 174307
 Edge 13 => 2314073
 ⋮
 Edge 2317162 => 2184961
 Edge 2317163 => 76865
 Edge 2317164 => 41358
 Edge 2317165 => 1334851
 Edge 2317166 => 1480
 Edge 2317167 => 598527
 Edge 2317168 => 145805
 Edge 2317169 => 120969
 Edge 2317170 => 48151
 Edge 2317171 => 16034
 Edge 2317172 => 166094
 Edge 2317173 => 2317172

Write out methodology for writing graph to disk and then re-importing into neo4j

Learn to visualize large graphs in Bloom

In [179]:
unique(ncbi_taxonomy.vprops[v][:division_cde] => ncbi_taxonomy.vprops[v][:division_name] for v in LightGraphs.vertices(ncbi_taxonomy))

12-element Array{Pair{SubString{String},SubString{String}},1}:
 "UNA" => "Unassigned"
 "BCT" => "Bacteria"
 "ENV" => "Environmental samples"
 "SYN" => "Synthetic and Chimeric"
 "PLN" => "Plants and Fungi"
 "INV" => "Invertebrates"
 "VRT" => "Vertebrates"
 "MAM" => "Mammals"
 "PRI" => "Primates"
 "ROD" => "Rodents"
 "VRL" => "Viruses"
 "PHG" => "Phages"

In [180]:
unique(ncbi_taxonomy.vprops[v][:division_cde] => ncbi_taxonomy.vprops[v][:division_name] for v in LightGraphs.vertices(ncbi_taxonomy))

12-element Array{Pair{SubString{String},SubString{String}},1}:
 "UNA" => "Unassigned"
 "BCT" => "Bacteria"
 "ENV" => "Environmental samples"
 "SYN" => "Synthetic and Chimeric"
 "PLN" => "Plants and Fungi"
 "INV" => "Invertebrates"
 "VRT" => "Vertebrates"
 "MAM" => "Mammals"
 "PRI" => "Primates"
 "ROD" => "Rodents"
 "VRL" => "Viruses"
 "PHG" => "Phages"

In [181]:
column_names = Set(k for vertex in LightGraphs.vertices(ncbi_taxonomy) for k in keys(ncbi_taxonomy.vprops[vertex]))
column_names = sort(collect(column_names))
# column_names = filter(x -> string(x) != "in-part", column_names)

18-element Array{Symbol,1}:
 :acronym
 :authority
 :blast_name
 :common_name
 :division_cde
 :division_id
 :division_name
 :equivalent_name
 :genbank_acronym
 :genbank_common_name
 :genbank_synonym
 :in_part
 :includes
 :rank
 :scientific_name
 :synonym
 :tax_id
 :type_material

In [182]:
open("$(DIR)/ncbi_taxonomy.nodes.tsv", "w") do io
    header = ["node", string.(column_names)...]
    println(io, join(header, '\t'))
    for vertex in collect(LightGraphs.vertices(ncbi_taxonomy))
        fields = String[]
        for k in column_names
            field = get(ncbi_taxonomy.vprops[vertex], k, "")
            field = string.(field)
            if field isa Array
                field = join(field, ';')
            end
            push!(fields, field)
        end
        row = ["$(vertex)", fields...]
        println(io, join(row, '\t'))
    end
end

In [235]:
collect(LightGraphs.edges(ncbi_taxonomy))[1:10]

10-element Array{LightGraphs.SimpleGraphs.SimpleEdge{Int64},1}:
 Edge 1 => 1
 Edge 2 => 100805
 Edge 3 => 280007
 Edge 4 => 3
 Edge 5 => 14946
 Edge 6 => 1384594
 Edge 7 => 1338
 Edge 8 => 165190
 Edge 9 => 8
 Edge 10 => 14787

In [238]:
open("$(DIR)/ncbi_taxonomy.edges.tsv", "w") do io
    header = ["src", "dst"]
    println(io, join(header, '\t'))
    for edge in collect(LightGraphs.edges(ncbi_taxonomy))
        src_tax_id = ncbi_taxonomy.vprops[edge.src][:tax_id]
        dst_tax_id = ncbi_taxonomy.vprops[edge.dst][:tax_id]
#         @show edge.src, 
#         @show edge.dst, 
        println(io, join(string.([src_tax_id, dst_tax_id]), "\t"))
    end
end

In [100]:
# LightGraphs.savegraph("$(DIR)/ncbi_taxonomy.graphml", LightGraphs.SimpleGraphFromIterator(LightGraphs.edges(ncbi_taxonomy)), "graph", GraphIO.GraphML.GraphMLFormat())

1

In [101]:
# LightGraphs.savegraph("$(DIR)/ncbi_taxonomy.gml", LightGraphs.SimpleGraphFromIterator(LightGraphs.edges(ncbi_taxonomy)), "graph", GraphIO.GML.GMLFormat())

1

In [102]:
# LightGraphs.savegraph("$(DIR)/ncbi_taxonomy.meta.graphml", ncbi_taxonomy, "graph", GraphIO.GraphML.GraphMLFormat())

1

In [103]:
# LightGraphs.savegraph("$(DIR)/ncbi_taxonomy.meta.gml", ncbi_taxonomy, "graph", GraphIO.GML.GMLFormat())

1

In [110]:
function list_databases()
    io = open(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database system --format auto "show databases"`)
    return DataFrames.DataFrame(uCSV.read(io, header=1, quotes='"', encodings=Dict("FALSE" => false, "TRUE" => true))...)
end

list_databases (generic function with 1 method)

In [111]:
function create_database(database_id)
    current_databases = list_databases()
    if database_id in current_databases[!, "name"]
        return
    else
        run(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database system --format auto "create database $(database_id)"`)
    end
end

create_database (generic function with 1 method)

In [112]:
function cypher(database_id, cmd)
    run(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database $(database_id) --format auto $(cmd)`)
end

cypher (generic function with 1 method)

In [116]:
list_databases()

,name,address,role,requestedStatus,currentStatus,error
,String,String,String,String,String,String
1,dataset,localhost:7687,standalone,online,online,
2,neo4j,localhost:7687,standalone,online,online,
3,panmetagenome,localhost:7687,standalone,online,online,
4,system,localhost:7687,standalone,online,online,
5,taxid230604,localhost:7687,standalone,online,online,
6,test,localhost:7687,standalone,online,online,


In [207]:
database_id = "ncbitaxonomy"

"ncbitaxonomy"

In [200]:
ENV["PATH"]

"/Users/cameronprybol/Software/cypher-shell:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Users/cameronprybol/miniconda/bin:/Users/cameronprybol/.gem/ruby/2.6.0/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin"

In [208]:
# clear a database
# @time cypher(database_id, "MATCH (t:Taxonomy) DETACH DELETE t")
# delete a database
# cypher("system", "DROP DATABASE $(database_id)")

Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database system --format auto 'DROP DATABASE ncbitaxonomy'`, ProcessExited(0))

In [209]:
create_database(database_id)

Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database system --format auto 'create database ncbitaxonomy'`, ProcessExited(0))

In [210]:
@time cypher(database_id, "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.tax_id IS UNIQUE")
@time cypher(database_id, "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.`scientific name` IS UNIQUE")

  1.245018 seconds (134 allocations: 7.984 KiB)
  1.235412 seconds (251 allocations: 17.656 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto 'CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.\`scientific name\` IS UNIQUE'`, ProcessExited(0))

In [213]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
MERGE (t:Taxonomy {tax_id: row.tax_id})
"""
@time cypher(database_id, cmd)

 70.661302 seconds (2.24 k allocations: 69.344 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.tax_id IS NOT NULL
MERGE (t:Taxonomy {tax_id: row.tax_id})
"`, ProcessExited(0))

In [217]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.scientific_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.scientific_name = row.scientific_name
"""
@time cypher(database_id, cmd)

 50.739977 seconds (1.63 k allocations: 51.844 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.scientific_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.scientific_name = row.scientific_name
"`, ProcessExited(0))

In [218]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.division_cde IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.division_cde = row.division_cde
"""
@time cypher(database_id, cmd)

 47.722319 seconds (1.54 k allocations: 49.016 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.division_cde IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.division_cde = row.division_cde
"`, ProcessExited(0))

In [219]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.division_id IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.division_id = row.division_id
"""
@time cypher(database_id, cmd)

 48.413401 seconds (1.57 k allocations: 49.812 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.division_id IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.division_id = row.division_id
"`, ProcessExited(0))

In [220]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.division_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.division_name = row.division_name
"""
@time cypher(database_id, cmd)

 50.492626 seconds (1.63 k allocations: 51.359 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.division_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.division_name = row.division_name
"`, ProcessExited(0))

In [221]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.rank IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.rank = row.rank
"""
@time cypher(database_id, cmd)

 49.657106 seconds (1.60 k allocations: 50.828 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.rank IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.rank = row.rank
"`, ProcessExited(0))

In [222]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.acronym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.acronym = row.acronym
"""
@time cypher(database_id, cmd)

 24.969174 seconds (852 allocations: 28.781 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.acronym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.acronym = row.acronym
"`, ProcessExited(0))

In [223]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.in_part IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.in_part = row.in_part
"""
@time cypher(database_id, cmd)

 22.704195 seconds (783 allocations: 27.047 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.in_part IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.in_part = row.in_part
"`, ProcessExited(0))

In [224]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.includes IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.includes = row.includes
"""
@time cypher(database_id, cmd)

 41.095390 seconds (1.34 k allocations: 42.953 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.includes IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.includes = row.includes
"`, ProcessExited(0))

In [225]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.common_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.common_name = row.common_name
"""
@time cypher(database_id, cmd)

 32.695385 seconds (1.09 k allocations: 35.859 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.common_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.common_name = row.common_name
"`, ProcessExited(0))

In [226]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.genbank_common_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.genbank_common_name = row.genbank_common_name
"""
@time cypher(database_id, cmd)

 39.209667 seconds (1.28 k allocations: 41.469 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.genbank_common_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.genbank_common_name = row.genbank_common_name
"`, ProcessExited(0))

In [227]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.blast_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.blast_name = row.blast_name
"""
@time cypher(database_id, cmd)

 20.754020 seconds (726 allocations: 25.234 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.blast_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.blast_name = row.blast_name
"`, ProcessExited(0))

In [228]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.synonym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.synonym = row.synonym
"""
@time cypher(database_id, cmd)

 42.617695 seconds (1.39 k allocations: 44.516 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.synonym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.synonym = row.synonym
"`, ProcessExited(0))

In [229]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.genbank_synonym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.genbank_synonym = row.genbank_synonym
"""
@time cypher(database_id, cmd)

 24.485144 seconds (837 allocations: 28.469 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.genbank_synonym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.genbank_synonym = row.genbank_synonym
"`, ProcessExited(0))

In [230]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.type_material IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.type_material = row.type_material
"""
@time cypher(database_id, cmd)

 41.317321 seconds (1.35 k allocations: 43.406 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.type_material IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.type_material = row.type_material
"`, ProcessExited(0))

In [231]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.authority IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.authority = row.authority
"""
@time cypher(database_id, cmd)

 44.953062 seconds (1.46 k allocations: 46.312 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.authority IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.authority = row.authority
"`, ProcessExited(0))

In [232]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.genbank_acronym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.genbank_acronym = row.genbank_acronym
"""
@time cypher(database_id, cmd)

 22.311734 seconds (774 allocations: 26.859 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.genbank_acronym IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.genbank_acronym = row.genbank_acronym
"`, ProcessExited(0))

In [233]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.nodes.tsv")' AS row
FIELDTERMINATOR '\t'
WITH row WHERE row.equivalent_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.equivalent_name = row.equivalent_name
"""
@time cypher(database_id, cmd)

 39.837954 seconds (1.31 k allocations: 41.938 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.nodes.tsv' AS row
FIELDTERMINATOR '	'
WITH row WHERE row.equivalent_name IS NOT NULL
MATCH (t:Taxonomy {tax_id: row.tax_id})
SET t.equivalent_name = row.equivalent_name
"`, ProcessExited(0))

In [241]:
cmd = 
"""
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file://$("$(DIR)/ncbi_taxonomy.edges.tsv")' AS row
FIELDTERMINATOR '\t'
MATCH (src:Taxonomy {tax_id: row.src})
MATCH (dst:Taxonomy {tax_id: row.dst})
MERGE (src)-[p:PARENT]->(dst)
"""
@time cypher(database_id, cmd)

 95.787084 seconds (2.99 k allocations: 91.219 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password password --database ncbitaxonomy --format auto "USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM
'file:///Users/cameronprybol/2021-03-21-ncbi-taxonomy/ncbi_taxonomy.edges.tsv' AS row
FIELDTERMINATOR '	'
MATCH (src:Taxonomy {tax_id: row.src})
MATCH (dst:Taxonomy {tax_id: row.dst})
MERGE (src)-[p:PARENT]->(dst)
"`, ProcessExited(0))